In [5]:
import findspark as fs
fs.init()

In [13]:
import os
os.environ["JAVA_HOME"]="C:\Java\jdk1.8.0_241"

In [14]:
import sparknlp 

spark = sparknlp.start()


# from sparknlp.pretrained import PretrainedPipeline
# pipeline = PretrainedPipeline('recognize_entities_dl', 'en')
# result = pipeline.annotate('Harry Potter is a great movie')
# print(result['entities']) 

recognize_entities_dl download started this may take some time.
Approx size to download 159 MB
[OK!]
['Harry Potter']


In [17]:
import pyspark
from pyspark.sql.functions import col,when,size,min, max,length
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder \
    .master('local[*]')\
    .appName('topic_modelling')\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5")\
    .getOrCreate()

In [19]:
df_parquet = spark.read.parquet("part-00000-a8a717e8-bf83-42e3-a0b7-084971053d99-c000.snappy.parquet")

In [20]:
# create a column and put in the detected language code
from langdetect import detect
getCode = lambda x:detect(x)
sent = "this is eng"
getCode(sent)

'en'

In [21]:
# easier method to convert to pandas then apply then function
df_parquet_pd = df_parquet.toPandas()
df_parquet_pd['lang_code'] = df_parquet_pd['description'].apply(getCode)
df_parquet_pd

,app_name,bundle,description,genres,os_platform,lang_code
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en
...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en


In [22]:
# load with only English data
df_parquet_en = df_parquet_pd.loc[df_parquet_pd['lang_code']=='en']
df_parquet_en.shape

(686, 6)

In [24]:
df_spark = spark.createDataFrame(df_parquet_en)

In [25]:
# Step 1. Text cleasing with punctuations
from pyspark.sql.functions import regexp_replace
REGEX = '[_,\\-.!?@#$%^&*+/\d]'
df_spark = df_spark.withColumn("description_clean",regexp_replace(df_spark.description,REGEX,' '))
df_spark.show()

+--------------------+--------------------+--------------------+--------------------+-----------+---------+--------------------+
|            app_name|              bundle|         description|              genres|os_platform|lang_code|   description_clean|
+--------------------+--------------------+--------------------+--------------------+-----------+---------+--------------------+
|Black Box - Movie...|           958957112|Black Box Movie L...|[Entertainment, M...|        IOS|       en|Black Box Movie L...|
|  Knife Throwing Max|          1491396285|you will have fun...|[Games_Sports, Ga...|        IOS|       en|you will have fun...|
|Lidow - Photo Edi...|           894532288|layout/grid/colla...|     [Photo & Video]|        IOS|       en|layout grid colla...|
|Vidstitch Frames ...|           712908978|■ #Vidstitch
■ Fe...|[Photo & Video, S...|        IOS|       en|■  Vidstitch
■ Fe...|
|Where's my Cat? -...|          1259238703|The cat is hiding...|[Entertainment, G...|        IOS|

In [28]:
from nltk.stem.porter import *
# Instantiate stemmer object
stemmer = PorterStemmer()

# Quick test of the stemming function
tokens = ["thanks", "its", "proverbially", "unexpected", "running"]
for t in tokens:
  print(stemmer.stem(t))

In [16]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pyspark.ml import Pipeline

stemmer = Stemmer()
stemmer.setInputCols(["description_clean"]).setOutputCol("description_stem")

Stemmer_05ee9bbfb2e7

In [27]:
df_spark = stemmer.transform(df_spark)

IllegalArgumentException: "requirement failed: Wrong or missing inputCols annotators in Stemmer_05ee9bbfb2e7.\n\nCurrent inputCols: description_clean. Dataset's columns:\n(column_name=app_name,is_nlp_annotator=false)\n(column_name=bundle,is_nlp_annotator=false)\n(column_name=description,is_nlp_annotator=false)\n(column_name=genres,is_nlp_annotator=false)\n(column_name=os_platform,is_nlp_annotator=false)\n(column_name=lang_code,is_nlp_annotator=false)\n(column_name=description_clean,is_nlp_annotator=false).\nMake sure such annotators exist in your pipeline, with the right output names and that they have following annotator types: token"